# **Step-by-Step Guide: Connecting NAS MariaDB to AWS for Fast Data Retrieval**
---
### **📌 Scenario:**
- **Your live database (`my_database`) is on NAS MariaDB**.
- **AWS is used for faster query execution** while keeping data on NAS.
- **Excel connects to AWS for data retrieval** instead of querying NAS directly.

---

## **🔹 Step 1: Enable Live Remote Access to NAS MariaDB**
AWS services need access to MariaDB running on NAS.

### **1️⃣ Enable MariaDB Remote Access on NAS**
#### **For Synology NAS:**
1. Open **DSM (DiskStation Manager)**.
2. Go to **Control Panel → MariaDB 10**.
3. **Enable TCP/IP connections** (uncheck "Only allow local connections").
4. Restart MariaDB.

#### **For Linux-Based NAS (via SSH):**
1. **Connect to NAS via SSH**:
   ```bash
   ssh user@nas-ip-address
   ```
2. **Edit MariaDB Configuration**:
   ```bash
   sudo nano /etc/mysql/mariadb.conf.d/50-server.cnf
   ```
3. **Modify `bind-address`**:
   ```
   bind-address = 0.0.0.0
   ```
4. **Save and Restart MariaDB**:
   ```bash
   sudo systemctl restart mariadb
   ```

✅ **MariaDB is now accessible remotely.**

---

### **2️⃣ Allow AWS EC2 to Connect to NAS**
#### **For Synology NAS:**
1. Open **DSM → Control Panel → Security → Firewall**.
2. **Create a new rule**:
   - **Protocol:** TCP
   - **Port:** 3306
   - **Source:** **AWS EC2 Public IP** (or `0.0.0.0/0` for testing)

#### **For Linux-Based NAS:**
1. **Open MySQL Port (3306)**:
   ```bash
   sudo ufw allow 3306/tcp
   ```
2. **Check Firewall Status**:
   ```bash
   sudo ufw status
   ```

✅ **AWS can now connect to NAS MariaDB.**

---

## **🔹 Step 2: Launch AWS EC2 for Fast Querying**
Since AWS cannot directly query NAS, **AWS EC2 acts as a database proxy**.

### **1️⃣ Launch an EC2 Instance**
1. **Log in to AWS Console** → Navigate to **EC2**.
2. Click **Launch Instance**.
3. **Choose Amazon Linux 2** (or Ubuntu).
4. Select **Instance Type** → Choose **t3.micro** (Free Tier eligible).
5. **Configure Security Group**:
   - **Allow MySQL (3306)** from **NAS IP**.
   - **Allow SSH (22)** from **your IP**.
6. **Click Launch** and **Download Key Pair**.

### **2️⃣ Connect to EC2**
1. **Open Terminal** and Run:
   ```bash
   ssh -i your-key.pem ec2-user@aws-ec2-ip
   ```

### **3️⃣ Install MySQL Client on EC2**
1. **Run on EC2 Terminal**:
   ```bash
   sudo yum install mariadb -y   # Amazon Linux
   sudo apt install mariadb-client -y  # Ubuntu
   ```

### **4️⃣ Test Connection from EC2 to NAS**
1. **Run on EC2**:
   ```bash
   mysql -h <NAS_IP> -u <DB_USER> -p
   ```
2. Enter **NAS MariaDB Password**.
3. **Run a test query**:
   ```sql
   USE my_database;
   SELECT * FROM search_terms ORDER BY created_at DESC LIMIT 10;
   ```

✅ **AWS EC2 is now connected to NAS MariaDB in real-time.**

---

## **🔹 Step 3: Optimize AWS Query Performance**
Even though NAS is primary storage, AWS can **cache & speed up queries**.

### **Option 1: Use AWS RDS Proxy for Caching**
1. **Go to AWS Console → RDS → Create RDS Proxy**.
2. Select:
   - **Database Engine**: MariaDB.
   - **Backend**: Your **NAS MariaDB**.
3. **Use the RDS Proxy Endpoint** in your queries:
   ```bash
   mysql -h <RDS_PROXY_ENDPOINT> -u <DB_USER> -p
   ```

✅ **AWS caches NAS queries for faster execution.**

---

### **Option 2: Use AWS S3 + Athena for Read-Only Queries**
For **Excel reports**, store `search_terms` in **AWS S3** & query via **Athena**.

1. **Export NAS Data to CSV Periodically**:
   ```sql
   SELECT * FROM search_terms
   INTO OUTFILE '/nas/data/search_terms.csv'
   FIELDS TERMINATED BY ','
   LINES TERMINATED BY '\n';
   ```
2. **Sync NAS CSV Files to AWS S3**:
   ```bash
   aws s3 sync /nas/data/ s3://your-bucket-name/
   ```
3. **Query Using Amazon Athena**:
   - Open **AWS Console → Athena**.
   - **Create a table** pointing to **S3 CSV files**.
   - Run **SQL queries in Athena**.

✅ **Athena provides near-instant query results without overloading NAS.**

---

## **🔹 Step 4: Connect Excel to AWS for Fast Data Retrieval**
Instead of slow queries on NAS, **Excel connects to AWS**.

### **Option 1: Connect Excel to AWS EC2 (Proxy to NAS)**
1. **Install MySQL ODBC Driver**:  
   👉 [Download Here](https://dev.mysql.com/downloads/connector/odbc/)
2. **Configure ODBC Data Source**:
   - **Server**: **AWS EC2 IP** (which connects to NAS)
   - **Port**: 3306
   - **Username/Password**: NAS DB credentials
3. **Import Data to Excel**:
   - Open **Excel → Data → Get Data → From ODBC**.
   - Select your ODBC connection.

✅ **Excel retrieves NAS data via AWS EC2.**

---

### **Option 2: Connect Excel to AWS Athena (Read-Only Reports)**
1. **Install Athena ODBC Driver**.
2. **Configure ODBC Data Source**:
   - **Server**: `athena.us-east-1.amazonaws.com`
   - **AWS Access Key & Secret** (IAM Role)
   - **Query Source**: **S3 Data**
3. **Import Athena Queries to Excel**:
   - **Excel → Data → Get Data → From ODBC**.

✅ **Excel retrieves NAS data via AWS Athena, which is much faster.**

---

## **🔹 Step 5: Automate Data Sync (NAS → AWS)**
Keep AWS data **up-to-date** using cron jobs.

### **1️⃣ Automate CSV Export from NAS**
1. Open **SSH Terminal on NAS**.
2. Run:
   ```bash
   crontab -e
   ```
3. Add:
   ```
   0 2 * * * mysql -u root -p'yourpassword' -e "SELECT * FROM search_terms WHERE created_at >= CURDATE() - INTERVAL 1 DAY INTO OUTFILE '/nas/data/daily_search_terms.csv' FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n';"
   ```

✅ **Exports fresh data every day.**

---

### **2️⃣ Automate AWS S3 Sync**
1. Create a sync script **`sync_to_s3.sh`**:
   ```bash
   #!/bin/bash
   aws s3 sync /nas/data/ s3://your-bucket-name/
   ```
2. Make it executable:
   ```bash
   chmod +x sync_to_s3.sh
   ```
3. Schedule in **crontab**:
   ```
   30 2 * * * /path/to/sync_to_s3.sh
   ```

✅ **AWS gets updated data daily.**

---

# **🔹 Final Workflow Summary**
| **Step** | **Solution** |
|----------|-------------|
| **Store Live Data** | NAS MariaDB (`my_database.search_terms`) |
| **Retrieve Live Data** | AWS EC2 (Proxy to NAS) |
| **Optimize Queries** | AWS RDS Proxy |
| **Fast Read-Only Reports** | AWS S3 + Athena |
| **Connect Excel** | ODBC to AWS |
| **Automate Sync** | Cron job & AWS CLI |

✅ **Your NAS database remains the primary storage, while AWS speeds up retrieval for Excel users!** 🚀

Would you like Python scripts for automation? 😊